Preparing on-street parking data - converting polygons into matchable points

In [1]:
import warnings
warnings.filterwarnings('ignore')


import os
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = 'db'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [2]:
parking = snman.io.import_geofile_to_gdf(
    "C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/Parkflächen Erhebung Stadt+OSM.gpkg"
).query("typ in ['Parkstreifen', 'Zickzack']")
parking

,typ,einschraenkung,einschraenkung_details,kapazitaet,art_fahrzeugaufstellung,standort,subtyp,Quelle,geometry
0,Parkstreifen,NaN,NaN,1.0,Längsparker,NaN,Behindertenparkplatz,Erhebung Stadt,"POLYGON ((2774148.466 1254031.491, 2774151.054..."
1,Parkstreifen,NaN,NaN,4.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2774148.715 1254026.448, 2774149.753..."
2,Parkstreifen,NaN,NaN,4.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2774157.135 1254049.946, 2774158.843..."
3,Parkstreifen,NaN,NaN,10.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2774149.618 1254043.180, 2774153.155..."
4,Parkstreifen,NaN,NaN,6.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2774076.224 1253693.448, 2774077.816..."
...,...,...,...,...,...,...,...,...,...
769,Parkstreifen,NaN,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2772341.502 1254208.495, 2772339.326..."
770,Parkstreifen,NaN,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2772353.937 1254183.985, 2772355.797..."
771,Parkstreifen,NaN,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2772369.666 1254162.400, 2772371.208..."
790,Parkstreifen,Kurzparkzone 120min,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,"POLYGON ((2774462.239 1253586.891, 2774460.753..."


In [3]:
import numpy as np
import shapely as shp

def random_points_in_polygon(polygon, number):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = shp.Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return shp.MultiPoint(points)

parking.geometry = parking.apply(
    lambda row: random_points_in_polygon(row.geometry, row['kapazitaet']),
    axis=1
)

In [6]:
parking = parking.explode().reset_index(drop=True)
parking

,typ,einschraenkung,einschraenkung_details,kapazitaet,art_fahrzeugaufstellung,standort,subtyp,Quelle,geometry
0,Parkstreifen,NaN,NaN,1.0,Längsparker,NaN,Behindertenparkplatz,Erhebung Stadt,POINT (2774148.705 1254030.452)
1,Parkstreifen,NaN,NaN,4.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2774150.637 1254017.255)
2,Parkstreifen,NaN,NaN,4.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2774150.549 1254018.097)
3,Parkstreifen,NaN,NaN,4.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2774149.224 1254024.840)
4,Parkstreifen,NaN,NaN,4.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2774151.493 1254012.931)
...,...,...,...,...,...,...,...,...,...
877,Parkstreifen,NaN,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2772377.882 1254152.953)
878,Parkstreifen,NaN,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2772373.381 1254158.670)
879,Parkstreifen,Kurzparkzone 120min,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2774467.370 1253580.515)
880,Parkstreifen,Kurzparkzone 120min,NaN,3.0,Längsparker,NaN,NaN,Erhebung Stadt,POINT (2774466.606 1253583.055)


In [7]:
snman.io.export_gdf(
    parking,
    "C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/Parkflächen Erhebung Stadt+OSM_punkte.gpkg"
)